# Bayes Estimator

### <font color='orange'>Universidad Autónoma de Yucatán</font> _Facultad de Matemáticas_

**Teacher:** Dr. Arturo Espinosa Romero <[eromero@correo.uady.mx](mailto:eromero@correo.uady.mx)>

**Student:** Ing. Dayan Bravo Fraga <[dayan3847@gmail.com](mailto:dayan3847@gmail.com)>

# Practice 5: Extended Kalman Filter for Ball Tracking

## GitHub: [Practice 5](https://github.com/dayan3847/bayes_estimator/tree/main/practice5-ball_tracking)

In [1]:
import sympy as sp
import numpy as np

In [2]:
#@title Estado de sistema
X, Y, Z, = sp.symbols('X Y Z')
dX, dY, dZ = sp.symbols('\dot{X} \dot{Y} \dot{Z}')
ddX, ddY, ddZ = sp.symbols('\ddot{X} \ddot{Y} \ddot{Z}')
# El estado esta compuesto por las coordenadas reales y las velocidades
XX = sp.Matrix([
    [X],
    [Y],
    [Z],
    [dX],
    [dY],
    [dZ],
    [ddX],
    [ddY],
    [ddZ],
])

sp.Eq(sp.Symbol('\mathbb{X}'), XX, evaluate=False)

Eq(\mathbb{X}, Matrix([
[       X],
[       Y],
[       Z],
[ \dot{X}],
[ \dot{Y}],
[ \dot{Z}],
[\ddot{X}],
[\ddot{Y}],
[\ddot{Z}]]))

In [3]:
sp.latex(sp.Eq(sp.Symbol('\mathbb{X}'), XX, evaluate=False))

'\\mathbb{X} = \\left[\\begin{matrix}X\\\\Y\\\\Z\\\\\\dot{X}\\\\\\dot{Y}\\\\\\dot{Z}\\\\\\ddot{X}\\\\\\ddot{Y}\\\\\\ddot{Z}\\end{matrix}\\right]'

In [14]:
#@title Matriz A "transitionMatrix"
dt = sp.Symbol('\Delta t')
dt2_2 = dt ** 2 / 2
# Esta seria la matrix de transformacion de estado
AA = sp.Matrix([
    [1, 0, 0, dt, 0, 0, dt2_2, 0, 0],
    [0, 1, 0, 0, dt, 0, 0, dt2_2, 0],
    [0, 0, 1, 0, 0, dt, 0, 0, dt2_2],
    [0, 0, 0, 1, 0, 0, dt, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, dt, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, dt],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
])
sp.Eq(sp.Symbol('\mathbb{A}'), AA, evaluate=False)

Eq(\mathbb{A}, Matrix([
[1, 0, 0, \Delta t,        0,        0, \Delta t**2/2,             0,             0],
[0, 1, 0,        0, \Delta t,        0,             0, \Delta t**2/2,             0],
[0, 0, 1,        0,        0, \Delta t,             0,             0, \Delta t**2/2],
[0, 0, 0,        1,        0,        0,      \Delta t,             0,             0],
[0, 0, 0,        0,        1,        0,             0,      \Delta t,             0],
[0, 0, 0,        0,        0,        1,             0,             0,      \Delta t],
[0, 0, 0,        0,        0,        0,             1,             0,             0],
[0, 0, 0,        0,        0,        0,             0,             1,             0],
[0, 0, 0,        0,        0,        0,             0,             0,             1]]))

In [15]:
#@title "Z" Medicion
x, y, r = sp.symbols('x y r')
dx, dy, dr = sp.symbols('\dot{x} \dot{y} \dot{r}')
ddx, ddy, ddr = sp.symbols('\ddot{x} \ddot{y} \ddot{r}')
# La medicion esta compuesta por las coordenadas de la camara y el tamanno del radio (en metros)
ZZ = sp.Matrix([
    [x],
    [y],
    [r],
    [dx],
    [dy],
    [dr],
    [ddx],
    [ddy],
    [ddr],
])
sp.Eq(sp.Symbol('\mathbb{Z}'), ZZ, evaluate=False)

Eq(\mathbb{Z}, Matrix([
[       x],
[       y],
[       r],
[ \dot{x}],
[ \dot{y}],
[ \dot{r}],
[\ddot{x}],
[\ddot{y}],
[\ddot{r}]]))

In [16]:
# @title Funcion h
Rm = sp.Symbol('Rm')  # Radio real de la pelota en metros
h_x = sp.Matrix([
    [X / Z],  # x
    [Y / Z],  # y
    [Rm / Z],  # r
    [-X * dZ / Z ** 2 + dX / Z],  # dx
    [-Y * dZ / Z ** 2 + dY / Z],  # dy
    [-Rm * dZ / Z ** 2],  # dr
    [(-X * (ddZ - 2 * dZ ** 2 / Z) / Z + ddX - 2 * dX * dZ / Z) / Z],  # ddx
    [(-Y * (ddZ - 2 * dZ ** 2 / Z) / Z + ddY - 2 * dY * dZ / Z) / Z],  # ddy
    [-Rm * (ddZ - 2 * dZ ** 2 / Z) / Z ** 2],  # ddr
])
sp.Eq(sp.Function('h')(sp.Symbol('\mathbb{X}')), h_x, evaluate=False)

Eq(h(\mathbb{X}), Matrix([
[                                                                  X/Z],
[                                                                  Y/Z],
[                                                                 Rm/Z],
[                                          -X*\dot{Z}/Z**2 + \dot{X}/Z],
[                                          -Y*\dot{Z}/Z**2 + \dot{Y}/Z],
[                                                     -Rm*\dot{Z}/Z**2],
[(-X*(\ddot{Z} - 2*\dot{Z}**2/Z)/Z + \ddot{X} - 2*\dot{X}*\dot{Z}/Z)/Z],
[(-Y*(\ddot{Z} - 2*\dot{Z}**2/Z)/Z + \ddot{Y} - 2*\dot{Y}*\dot{Z}/Z)/Z],
[                                 -Rm*(\ddot{Z} - 2*\dot{Z}**2/Z)/Z**2]]))

In [17]:
# @title Matriz H es el Jacobian de h respecto al estado
HH = h_x.jacobian(XX)
sp.Eq(sp.Symbol('\mathbb{H}x'), HH, evaluate=False)

Eq(\mathbb{H}x, Matrix([
[                              1/Z,                                 0,                                                                                                                                                          -X/Z**2,               0,               0,                                  0,   0,   0,        0],
[                                0,                               1/Z,                                                                                                                                                          -Y/Z**2,               0,               0,                                  0,   0,   0,        0],
[                                0,                                 0,                                                                                                                                                         -Rm/Z**2,               0,               0,                                  0,   0,   0,        0],
[  

In [20]:
HH[6, 2]

(X*(\ddot{Z} - 2*\dot{Z}**2/Z)/Z**2 - 2*X*\dot{Z}**2/Z**3 + 2*\dot{X}*\dot{Z}/Z**2)/Z - (-X*(\ddot{Z} - 2*\dot{Z}**2/Z)/Z + \ddot{X} - 2*\dot{X}*\dot{Z}/Z)/Z**2

In [25]:
print(
    HH[6, 2]
    .subs(dX, sp.Symbol('dX'))
    .subs(dY, sp.Symbol('dY'))
    .subs(dZ, sp.Symbol('dZ'))
    .subs(ddX, sp.Symbol('ddX'))
    .subs(ddY, sp.Symbol('ddY'))
    .subs(ddZ, sp.Symbol('ddZ'))
)

(X*(ddZ - 2*dZ**2/Z)/Z**2 - 2*X*dZ**2/Z**3 + 2*dX*dZ/Z**2)/Z - (-X*(ddZ - 2*dZ**2/Z)/Z + ddX - 2*dX*dZ/Z)/Z**2


In [28]:
rows, cols = HH.shape
for r in range(rows):
    line = ''
    for c in range(cols):
        line += str(
            HH[r, c]
            .subs(dX, sp.Symbol('dX'))
            .subs(dY, sp.Symbol('dY'))
            .subs(dZ, sp.Symbol('dZ'))
            .subs(ddX, sp.Symbol('ddX'))
            .subs(ddY, sp.Symbol('ddY'))
            .subs(ddZ, sp.Symbol('ddZ'))
        ) + ', '
    print(line)

1/Z, 0, -X/Z**2, 0, 0, 0, 0, 0, 0, 
0, 1/Z, -Y/Z**2, 0, 0, 0, 0, 0, 0, 
0, 0, -Rm/Z**2, 0, 0, 0, 0, 0, 0, 
-dZ/Z**2, 0, 2*X*dZ/Z**3 - dX/Z**2, 1/Z, 0, -X/Z**2, 0, 0, 0, 
0, -dZ/Z**2, 2*Y*dZ/Z**3 - dY/Z**2, 0, 1/Z, -Y/Z**2, 0, 0, 0, 
0, 0, 2*Rm*dZ/Z**3, 0, 0, -Rm/Z**2, 0, 0, 0, 
-(ddZ - 2*dZ**2/Z)/Z**2, 0, (X*(ddZ - 2*dZ**2/Z)/Z**2 - 2*X*dZ**2/Z**3 + 2*dX*dZ/Z**2)/Z - (-X*(ddZ - 2*dZ**2/Z)/Z + ddX - 2*dX*dZ/Z)/Z**2, -2*dZ/Z**2, 0, (4*X*dZ/Z**2 - 2*dX/Z)/Z, 1/Z, 0, -X/Z**2, 
0, -(ddZ - 2*dZ**2/Z)/Z**2, (Y*(ddZ - 2*dZ**2/Z)/Z**2 - 2*Y*dZ**2/Z**3 + 2*dY*dZ/Z**2)/Z - (-Y*(ddZ - 2*dZ**2/Z)/Z + ddY - 2*dY*dZ/Z)/Z**2, 0, -2*dZ/Z**2, (4*Y*dZ/Z**2 - 2*dY/Z)/Z, 0, 1/Z, -Y/Z**2, 
0, 0, 2*Rm*(ddZ - 2*dZ**2/Z)/Z**3 - 2*Rm*dZ**2/Z**4, 0, 0, 4*Rm*dZ/Z**3, 0, 0, -Rm/Z**2, 


In [5]:
sp.latex(sp.Function('h')(sp.Symbol('x')))

'h{\\left(x \\right)}'